In [3]:
# Necessary libraries
import os
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import utils
import keras
import imageio 
from PIL import Image
# Reading the data
train = pd.read_csv('./dataset/age_detection_train/train.csv')
# Image resizing of train data into single numpy array
temp = []
for img_name in train.ID:
    img_path = os.path.join('./dataset/age_detection_train/Train', img_name)
    img = imageio.imread(img_path)
    img = np.array(Image.fromarray(img).resize((32, 32))).astype('float32')    
    temp.append(img)
train_x = np.stack(temp)
# Normalizing the images
train_x = train_x / 255.
# Encoding the categorical variable to numeric
lb = LabelEncoder()
train_y = lb.fit_transform(train.Class)
train_y = tf.keras.utils.to_categorical(train_y)
# Specifying all the parameters we will be using in our network
input_num_units = (32, 32, 3)
hidden_num_units = 500
output_num_units = 3
epochs = 100
batch_size = 128


C:\Users\TanmoY\AppData\Local\Temp\ipykernel_15328\618740220.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(img_path)


First, let us define the network, as shown below:

In [4]:
# Defining the network
model = Sequential([
  InputLayer(input_shape=input_num_units),
  Flatten(),
  Dense(units=hidden_num_units, activation='relu'),
  Dense(units=output_num_units, activation='softmax'),
])


c:\Users\TanmoY\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Next, let us write a function to build a model for each optimizer and save the accuracy, loss, validation accuracy and validation loss for Tensor board visualization.

In [5]:
def models_with_different_optimizers(list_of_optimizers):    
    
    for i in range(len(list_of_optimizers)):        
        model.compile(loss='categorical_crossentropy',
                  optimizer=list_of_optimizers[i], # Learning rate and momentum can be passed inside optimizer
                  metrics=['accuracy'])
        # Traning the model and writing log files for TensorBoard in distinct directories        
        val = re.search('optimizers\..*\so', str(list_of_optimizers[i])).group(0)[11:][:-2] # Fetching optimizer name
        logdir = r'optims\\' + val # Each log file needs to be written in a distinct directory. (Mandatory)
        
        # Writing graph will take time. Hence, keeping it False.
        cb = keras.callbacks.TensorBoard(log_dir=logdir, write_graph=False)         
        print('Building model using '+ val + ' optimizer')
        history = model.fit(train_x, train_y, epochs=epochs, 
                           validation_split=0.2,
                           callbacks=[cb])
        print('Model built sucessfully.')
        print('')
# Listing the optimizers
optims = [keras.optimizers.Adam(), keras.optimizers.Adadelta(), 
          keras.optimizers.Adagrad(), keras.optimizers.RMSprop(), 
          keras.optimizers.SGD()]
# Calling the function
models_with_different_optimizers(optims)


Building model using adam.Adam optimizer
Epoch 1/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.5419 - loss: 1.0997 - val_accuracy: 0.6035 - val_loss: 0.8371
Epoch 2/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.6100 - loss: 0.8321 - val_accuracy: 0.6173 - val_loss: 0.8206
Epoch 3/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.6215 - loss: 0.8103 - val_accuracy: 0.6293 - val_loss: 0.8110
Epoch 4/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.6293 - loss: 0.8059 - val_accuracy: 0.6339 - val_loss: 0.8062
Epoch 5/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.6428 - loss: 0.7916 - val_accuracy: 0.6326 - val_loss: 0.8070
Epoch 6/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.6542 - loss: 0.7682 - val_accuracy: 0.6549 - val_loss: 0.7722
Epoch 7/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.6622 - loss: 0.7562 - val_accuracy: 0.6673 - val_loss: 0.7551
Epoch 8/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/